# Lab 8: Wolf-Goat-Cabbage

This notebook covers the basic search algorithms for our new problem, **Wolf-Goat-Cabbage**.

The search algorithms we will be showing first are 
- `breadth_first_search`
- `depth_first_search`
- `uniformed_cost_search`
- `depth_limited_search`
- `iterative_deepening`

"Wolf-Goat-Cabbage" is a classic logic puzzle that involves transporting a wolf, a goat, and a cabbage across a river, but with certain constraints to prevent conflicts.

#### Actors
- Boat for transporting
- Wolf
- Goat
- Cabbage

#### Environment
- The left river bank (Starting Zone)
- The River
- The right river bank (Goal Zone)

#### Objective
Your goal is to figure out a sequence of crossings that gets all 3 items safely to the other side of the river without any of them being eaten.


#### Rules
- Wolf cannot be left alone with Goat
- Goat cannot be left alone with Cabbage


In [1]:
class WGC_Node:
    incompatibilities = [
        ["c", "g", "w"],
        ["g", "w"],
        ["c", "g"]
    ]

    def __init__(self, west=["w", "c", "g"], east=[], boat_side=False, children=[]):
        self.west = west
        self.east = east
        self.boat_side = boat_side
        self.children = children

    def __str__(self):
        return str(self.west) + str(self.east) + ("Left" if not self.boat_side else "Right")

    def generate_children(self, previous_states, parent_map):
        children = []
        if not self.boat_side:
            for i in self.west:
                new_west = self.west[:]
                new_west.remove(i)
                new_east = self.east[:]
                new_east.append(i)
                if sorted(new_west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    children.append(child)
                    parent_map[child] = self
            if sorted(self.west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                children.append(child)
                parent_map[child] = self
        else:
            for i in self.east:
                new_west = self.west[:]
                new_west.append(i)
                new_east = self.east[:]
                new_east.remove(i)
                if sorted(new_east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    children.append(child)
                    parent_map[child] = self
            if sorted(self.east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                children.append(child)
                parent_map[child] = self
        self.children = children

    @staticmethod
    def state_in_previous(previous_states, west, east, boat_side):
        return any(
            sorted(west) == sorted(i.west) and
            sorted(east) == sorted(i.east) and
            boat_side == i.boat_side
            for i in previous_states
        )


def find_solution(root_node, use_bfs=False):
    '''
    Find a solution to the WGC Problem.
    use_bfs: False for DFS, True for BFS
    '''
    to_visit = [root_node]
    node = root_node
    previous_states = []
    parent_map = {root_node: None}
    while to_visit:
        node = to_visit.pop()
        if not WGC_Node.state_in_previous(previous_states, node.west, node.east, node.boat_side):
            previous_states.append(node)
        node.generate_children(previous_states, parent_map)
        if use_bfs:
            to_visit = node.children + to_visit
        else:
            to_visit = to_visit + node.children
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                node = parent_map[node]
            return solution
    return None

if __name__ == "__main__":
    root = WGC_Node()
    solution = find_solution(root, use_bfs=False)
    print("DFS solution = [", end='')
    for i in solution:
        print(i, '\b, ', end='')
    print("\b\b]")

    solution = find_solution(root, use_bfs=True)
    print("BFS solution = [", end='')
    for i in solution:
        print(i, '\b, ', end='')
    print("\b\b]")

DFS solution = [['w', 'c', 'g'][]Left , ['w', 'c']['g']Right , ['w', 'c']['g']Left , ['w']['g', 'c']Right , ['w', 'g']['c']Left , ['g']['c', 'w']Right , ['g']['c', 'w']Left , []['c', 'w', 'g']Right , ]
BFS solution = [['w', 'c', 'g'][]Left , ['w', 'c']['g']Right , ['w', 'c']['g']Left , ['w']['g', 'c']Right , ['w', 'g']['c']Left , ['g']['c', 'w']Right , ['g']['c', 'w']Left , []['c', 'w', 'g']Right , ]


In [2]:
import heapq

def uniform_cost_search(root_node):
    heap = [(0, id(root_node), root_node)]  # Priority queue with cost and node id
    visited = set()
    parent_map = {id(root_node): None}
    while heap:
        cost, _, node = heapq.heappop(heap)
        if id(node) in visited:
            continue
        visited.add(id(node))
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                parent_id = parent_map.get(id(node))
                node = parent_map.get(parent_id) if parent_id is not None else None
            return solution
        node.generate_children([], parent_map)
        for child in node.children:
            child_id = id(child)
            if child_id not in visited:
                heapq.heappush(heap, (cost + 1, child_id, child))
                parent_map[child_id] = id(node)
    return None

# Usage
solution = uniform_cost_search(root)
print("Uniform-Cost Search solution = [", end='')
for node in solution:
    print(node, '\b, ', end='')
print("\b\b]")


Uniform-Cost Search solution = [2773442103248 , []['c', 'w', 'g']Right , ]


In [3]:
def depth_limited_search(node, depth_limit, parent_map):
    if depth_limit == 0:
        return None
    if sorted(node.east) == ["c", "g", "w"]:
        solution = []
        while node is not None:
            solution = [node] + solution
            node = parent_map[node]
        return solution
    node.generate_children([], parent_map)
    for child in node.children:
        result = depth_limited_search(child, depth_limit - 1, parent_map)
        if result is not None:
            return result
    return None

# Usage
depth_limit = 10  # Set your desired depth limit
parent_map = {root: None}
solution = depth_limited_search(root, depth_limit, parent_map)
print("Depth-Limited Search solution with depth limit {} = [".format(depth_limit), end='')
for node in solution:
    print(node, '\b, ', end='')
print("\b\b]")


Depth-Limited Search solution with depth limit 10 = [['w', 'c', 'g'][]Left , ['w', 'c']['g']Right , ['w', 'c', 'g'][]Left , ['w', 'c']['g']Right , ['w', 'c']['g']Left , ['c']['g', 'w']Right , ['c', 'g']['w']Left , ['g']['w', 'c']Right , ['g']['w', 'c']Left , []['w', 'c', 'g']Right , ]


In [4]:
def iterative_deepening(root_node, max_depth):
    for depth_limit in range(1, max_depth + 1):
        parent_map = {root_node: None}
        solution = depth_limited_search(root_node, depth_limit, parent_map)
        if solution is not None:
            return solution
    return None

# Usage
max_depth = 20  # Set your desired maximum depth
solution = iterative_deepening(root, max_depth)
print("Iterative Deepening solution with max depth {} = [".format(max_depth), end='')
for node in solution:
    print(node, '\b, ', end='')
print("\b\b]")


Iterative Deepening solution with max depth 20 = [['w', 'c', 'g'][]Left , ['w', 'c']['g']Right , ['w', 'c']['g']Left , ['c']['g', 'w']Right , ['c', 'g']['w']Left , ['g']['w', 'c']Right , ['g']['w', 'c']Left , []['w', 'c', 'g']Right , ]
